# **Installation**

In [1]:
!pip install -q kaggle

In [2]:
!pip install \
    --extra-index-url=https://pypi.nvidia.com \
    cudf-cu12==24.10.* cuml-cu12==24.10.*

Looking in indexes: https://pypi.org/simple, https://pypi.nvidia.com
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 567.7/567.7 MB 1.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 836.6/836.6 MB 1.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 133.2/133.2 kB 12.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 kB 5.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 196.9/196.9 MB 10.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.1/2.1 MB 87.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 722.7/722.7 kB 49.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 511.7/511.7 kB 47.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 916.0/916.0 kB 49.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 63.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 242.6/242.6 kB 20.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━

In [3]:
!pip install category_encoders

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 82.0/82.0 kB 7.2 MB/s eta 0:00:00


In [4]:
!pip install catboost

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 98.7/98.7 MB 20.3 MB/s eta 0:00:00


# **Imports**

In [5]:
%load_ext cudf.pandas
import pandas as pd
from category_encoders import TargetEncoder

import numpy as np
import seaborn as sns
import pandas as pd
import matplotlib.pyplot as plt
from xgboost import XGBRegressor
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import mean_squared_error

pd.pandas.set_option('display.max_columns',None)

# Load Kaggle Dataset with:

In [6]:
import kagglehub
kagglehub.login()

Kaggle credentials set.
Kaggle credentials successfully validated.


In [8]:
odsc_2024_nvidia_hackathon_path = kagglehub.competition_download('odsc-2024-nvidia-hackathon')
print('Data source import complete')

100%|██████████| 3.72G/3.72G [02:48<00:00, 23.7MB/s]

Extracting files...


Data source import complete


# **Load Data** - Used pandas with cudf for parallel processing

In [10]:
train = pd.read_csv(f"{odsc_2024_nvidia_hackathon_path}/train.csv")[['trickortreat', 'kingofhalloween', 'mumming', 'noon', 'costumes',
       'celebrate', 'confectionary', 'predestinated', 'leprechaun', 'headless',
       'jacksantino', 'mystical', 'folklore', 'tarantula', 'paranormal',
       'northernlights', 'quirky', 'ectoplasm', 'jester', 'nightmare',
       'isolation', 'cobwebs', 'sweets', 'duskhour', 'bedford', 'propitiation',
       'ritesofpassage', 'kiddies', 'ghost', 'batman', 'applebobbing',
       'magical', 'h2o', 'xray', 'cackle', 'mummy', 'easter', 'fest',
       'washday', 'wardrobe', 'moonlight', 'theme', 'moon', 'week', 'parades',
       'imprisonment', 'worktime', 'haunt', 'frankenstein', 'october',
       'scoobydoo', 'rip', 'somerset', 'extravaganza', 'fortunetelling',
       'dracula', 'divination', 'kidnap', 'funeral', 'ween', 'nonconformism',
       'mischievous', 'mondays', 'batwings', 'fear', 'daysail', 'krakatoa',
       'egg', 'tshirt', 'night', 'apotropaicmagic', 'periodoftime', 'almanac',
       'dayspring', 'fortnight', 'dungeon', 'flagday', 'leapweek', 'ancestors',
       'lurking', 'sympatheticmagic', 'toys', 'lunacy', 'bewitching',
       'enchanting', 'casket', 'centiday', 'fire', 'interday', 'fogmachine',
       'holi', 'churchyard', 'monsterhunter', 'tabulatable', 'vampire','hallo','y']]

In [11]:
# train = pd.read_csv(f"{odsc_2024_nvidia_hackathon_path}/train.csv")#.sample(frac=0.50, random_state=42)

# **`Data Analysis`**

# **Feature Engineering**

In [12]:
train_x = train

train_x[['trickortreat_1', 'trickortreat_2']] = train_x['trickortreat'].str.split('_', expand=True)
train_x[['kingofhalloween_1', 'kingofhalloween_2']] = train_x['kingofhalloween'].str.split('_', expand=True)

# Drop original columns if not needed
train_x = train_x.drop(columns=['trickortreat','kingofhalloween'])

# **Target Encoding for Categorical data**

In [13]:
te = TargetEncoder()
encoders = {}
# Apply Target Encoding to each split column using the target variable
for col in ['trickortreat_1', 'trickortreat_2', 'kingofhalloween_1', 'kingofhalloween_2']:
    te = TargetEncoder()
    train_x[col] = te.fit_transform(train_x[col], train_x['y'])
    encoders[col] = te

print(train_x[['trickortreat_1', 'trickortreat_2', 'kingofhalloween_1', 'kingofhalloween_2']].head())

   trickortreat_1  trickortreat_2  kingofhalloween_1  kingofhalloween_2
0    42347.170756    42350.176677       42331.958544       42357.958762
1    42347.670434    42351.823577       42319.688544       42343.453836
2    42347.713021    42354.652862       42351.105099       42349.236474
3    42346.808867    42342.503325       42333.250757       42335.135550
4    42354.236507    42349.468489       42367.862562       42335.779101


In [14]:
train_x = train_x.drop(columns = ['y'])
train_y = train['y']

# **Filling missing values with mean for numerical and most frequent for categorical**

In [15]:
# Identify numerical columns (non-object types)
numerical_data_train = train_x.select_dtypes(include=['number']).columns.tolist()

In [16]:
# Initialize an empty dictionary to store fill values
fill_dict = {}

# Define a function to calculate the mean for numeric columns
def mean_value(col):
    return train_x[col].mean()

# Fill missing values for numerical and categorical columns
for col in numerical_data_train:
    if pd.api.types.is_numeric_dtype(train_x[col]):
        fill_dict[col] = mean_value(col)  # Only numeric columns
    else:
        print(f"Warning: {col} is not numeric. Skipping mean calculation.")


In [21]:
# Fill the missing values in the dataframe
train_x.fillna(fill_dict, inplace=True)

# **Build Model**

In [18]:
# Configure XGBRegressor with GPU support
Xgb = XGBRegressor(n_estimators = 400, max_depth=5, random_state  = 0, tree_method = "hist", device = "cuda")
Xgb.fit(train_x, train_y)


XGBRegressor(base_score=None, booster=None, callbacks=None,
             colsample_bylevel=None, colsample_bynode=None,
             colsample_bytree=None, device='cuda', early_stopping_rounds=None,
             enable_categorical=False, eval_metric=None, feature_types=None,
             gamma=None, grow_policy=None, importance_type=None,
             interaction_constraints=None, learning_rate=None, max_bin=None,
             max_cat_threshold=None, max_cat_to_onehot=None,
             max_delta_step=None, max_depth=5, max_leaves=None,
             min_child_weight=None, missing=nan, monotone_constraints=None,
             multi_strategy=None, n_estimators=400, n_jobs=None,
             num_parallel_tree=None, random_state=0, ...)

# **Prediction on train data**

In [19]:
pred_train =  Xgb.predict(train_x)

/usr/local/lib/python3.10/dist-packages/xgboost/core.py:158: UserWarning: [16:37:39] WARNING: /workspace/src/common/error_msg.cc:58: Falling back to prediction using DMatrix due to mismatched devices. This might lead to higher memory usage and slower performance. XGBoost is running on: cuda:0, while the input data is on: cpu.
Potential solutions:
- Use a data structure that matches the device ordinal in the booster.
- Set the device for booster before call to inplace_predict.

This warning will only be shown once.

  warnings.warn(smsg, UserWarning)


In [20]:

rmse = np.sqrt(mean_squared_error(train_y.to_numpy(), pred_train))
print("Train Root Mean Squared Error:", rmse)


Train Root Mean Squared Error: 638.7235748824622


In [22]:
train_x.columns

Index(['mumming', 'noon', 'costumes', 'celebrate', 'confectionary',
       'predestinated', 'leprechaun', 'headless', 'jacksantino', 'mystical',
       'folklore', 'tarantula', 'paranormal', 'northernlights', 'quirky',
       'ectoplasm', 'jester', 'nightmare', 'isolation', 'cobwebs', 'sweets',
       'duskhour', 'bedford', 'propitiation', 'ritesofpassage', 'kiddies',
       'ghost', 'batman', 'applebobbing', 'magical', 'h2o', 'xray', 'cackle',
       'mummy', 'easter', 'fest', 'washday', 'wardrobe', 'moonlight', 'theme',
       'moon', 'week', 'parades', 'imprisonment', 'worktime', 'haunt',
       'frankenstein', 'october', 'scoobydoo', 'rip', 'somerset',
       'extravaganza', 'fortunetelling', 'dracula', 'divination', 'kidnap',
       'funeral', 'ween', 'nonconformism', 'mischievous', 'mondays',
       'batwings', 'fear', 'daysail', 'krakatoa', 'egg', 'tshirt', 'night',
       'apotropaicmagic', 'periodoftime', 'almanac', 'dayspring', 'fortnight',
       'dungeon', 'flagday', 'leapw

# Testing

In [25]:
test = pd.read_csv(f"{odsc_2024_nvidia_hackathon_path}/test.csv")
test_x =test[['mumming', 'noon', 'costumes', 'celebrate', 'confectionary',
       'predestinated', 'leprechaun', 'headless', 'jacksantino', 'mystical',
       'folklore', 'tarantula', 'paranormal', 'northernlights', 'quirky',
       'ectoplasm', 'jester', 'nightmare', 'isolation', 'cobwebs', 'sweets',
       'duskhour', 'bedford', 'propitiation', 'ritesofpassage', 'kiddies',
       'ghost', 'batman', 'applebobbing', 'magical', 'h2o', 'xray', 'cackle',
       'mummy', 'easter', 'fest', 'washday', 'wardrobe', 'moonlight', 'theme',
       'moon', 'week', 'parades', 'imprisonment', 'worktime', 'haunt',
       'frankenstein', 'october', 'scoobydoo', 'rip', 'somerset',
       'extravaganza', 'fortunetelling', 'dracula', 'divination', 'kidnap',
       'funeral', 'ween', 'nonconformism', 'mischievous', 'mondays',
       'batwings', 'fear', 'daysail', 'krakatoa', 'egg', 'tshirt', 'night',
       'apotropaicmagic', 'periodoftime', 'almanac', 'dayspring', 'fortnight',
       'dungeon', 'flagday', 'leapweek', 'ancestors', 'lurking',
       'sympatheticmagic', 'toys', 'lunacy', 'bewitching', 'enchanting',
       'casket', 'centiday', 'fire', 'interday', 'fogmachine', 'holi',
       'churchyard', 'monsterhunter', 'tabulatable', 'vampire', 'hallo', 'trickortreat','kingofhalloween' ]]


test_x[['trickortreat_1', 'trickortreat_2']] = test_x['trickortreat'].str.split('_', expand=True)

# Split 'kingofhalloween' into two new columns, using n=2 to enforce two splits
test_x[['kingofhalloween_1', 'kingofhalloween_2']] = test_x['kingofhalloween'].str.split('_', expand=True, n=2)

print(test_x['kingofhalloween_2'].head())


# numerical_data_train = testx.select_dtypes(include=['number']).columns.tolist()

0    terrorless
1          <NA>
2         panic
3         genre
4    soundtrack
Name: kingofhalloween_2, dtype: object


In [26]:
# Drop original columns if not needed
test_x = test_x.drop(columns=['trickortreat','kingofhalloween'])
categorical_columns = ['trickortreat_1', 'trickortreat_2', 'kingofhalloween_1', 'kingofhalloween_2']

for col in categorical_columns:
    test_x[col] = encoders[col].transform(test_x[col])

In [27]:
# Initialize an empty dictionary to store fill values
fill_dict = {}

#Define a function to calculate the mean for numeric columns
def mean_value(col):
    return test_x[col].mean()

# Fill missing values for numerical and categorical columns
for col in numerical_data_train:
    if pd.api.types.is_numeric_dtype(test_x[col]):
        fill_dict[col] = mean_value(col)  # Only numeric columns
    else:
        print(f"Warning: {col} is not numeric. Skipping mean calculation.")


# Fill the missing values in the dataframe
test_x.fillna(fill_dict, inplace=True)

In [28]:
y_test_pred= Xgb.predict(test_x)

# **Create Submission file**

In [29]:
# Step 2: Create the submission DataFrame
submission = pd.DataFrame({ 'id': test['id'].apply(int), 'y': y_test_pred })
# Step 3: Save the submission file as a CSV
submission.to_csv('sample_submission.csv', index=False)